In [10]:
# Importamos las librerias necesarias.
import pandas as pd
import numpy as np

In [16]:
# Creamos un DataFrame de nuestros datos.
car_data = pd.read_csv('../data/cars.csv', sep=';')

# Mostramos la forma y los primeros registros de nuestro DataFrame.
print(car_data.shape)
car_data.head()

(58049, 25)


,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,REV_Garantia,Averia_grave,QUEJA_CAC,EDAD_COCHE,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad Cliente,Tiempo
0,CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,...,NO DATA,Averia muy grave,SI,2.08,2892,0,0,2,18,0
1,CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,...,SI,No,NO,2.45,1376,7187,0,2,53,0
2,CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,...,NO DATA,No,NO,2.51,1376,0,1,4,21,3
3,CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,...,SI,Averia muy grave,SI,2.43,2015,7256,1,4,48,5
4,CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,...,NO DATA,No,NO,2.30,1818,0,1,3,21,3


In [12]:
# Revisamos si hay valores nulos en nuestro DataFrame.
valores_nulos = car_data.isnull().sum()

# Mostramos los valores nulos.
valores_nulos

CODE                   0
PRODUCTO               0
TIPO_CARROCERIA        0
COMBUSTIBLE            0
Potencia_              0
TRANS                  0
FORMA_PAGO             0
ESTADO_CIVIL         890
GENERO               860
OcupaciOn              0
PROVINCIA              0
Campanna1              0
Campanna2              0
Campanna3              0
Zona _Renta        13178
REV_Garantia           0
Averia_grave           1
QUEJA_CAC              0
EDAD_COCHE             0
COSTE_VENTA            0
km_anno                0
Mas_1_coche            0
Revisiones             0
Edad Cliente           0
Tiempo                 0
dtype: int64

In [13]:
# Revisamos si hay valores duplicados en nuestro DataFrame.
duplicados = car_data.duplicated().sum()

# Mostramos los valores duplicados.
duplicados

0

In [14]:
# Revisamos los tipos de datos de nuestro DataFrame.
tipos_de_datos = car_data.dtypes

# Mostramos los tipos de datos.
tipos_de_datos

CODE                object
PRODUCTO            object
TIPO_CARROCERIA     object
COMBUSTIBLE         object
Potencia_           object
TRANS               object
FORMA_PAGO          object
ESTADO_CIVIL        object
GENERO              object
OcupaciOn           object
PROVINCIA           object
Campanna1           object
Campanna2           object
Campanna3           object
Zona _Renta         object
REV_Garantia        object
Averia_grave        object
QUEJA_CAC           object
EDAD_COCHE         float64
COSTE_VENTA          int64
km_anno              int64
Mas_1_coche          int64
Revisiones           int64
Edad Cliente         int64
Tiempo               int64
dtype: object